In [1]:
# import modules 
import matplotlib.cbook as cbook
from matplotlib_scalebar.scalebar import ScaleBar
import pandas as pd 
import geopandas as gpd 
import matplotlib.pyplot as plt 
import numpy as np
import PIL
import io
import geoplot 
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
## read the population data 
population = pd.read_excel('../../02_Data/population_2020.xlsx')
population

,NUTS_ID,nuts_name,Population
0,EU27_2020,European Union - 27 countries (from 2020),447319916
1,EU28,European Union - 28 countries (2013-2020),:
2,EU27_2007,European Union - 27 countries (2007-2013),:
3,BE,Belgium,11522440
4,BE1,Région de Bruxelles-Capitale/Brussels Hoofdste...,1223364
...,...,...,...
2785,PL62,NaN,2337769
2786,PL63,NaN,2337769
2787,PL33,NaN,1237369
2788,PL34,NaN,1179430


In [3]:
# read ESPON LOGO 
im = plt.imread('Extra/espon-eu-logo-vector.png') # insert local ESPON logo.

In [4]:
# make the base map 
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

In [5]:
fiji = world.query('name == "Fiji"')[['geometry']]

In [6]:
# Read shape file and associate data.
v3 = gpd.read_file('../../02_Data/01_GIS/01_ESPON Narrow MapKit NUTS2016/nuts-version2016-level3.shp')
## read the shape file 
shape = gpd.read_file('../../02_Data/01_GIS/02_Full file/NUTS_COVID.shp')
add = v3[v3['id'].str.contains('IE')].drop('name', axis =1)
add = add.to_crs(epsg = 4326)
add.rename(columns = {"id": 'NUTS_ID'}, inplace = True)
nor = gpd.read_file('../../02_Data/01_GIS/03_Norway_GIS/Norway.shp')
nor.rename(columns = {'UID': 'NUTS_ID'}, inplace = True)
nor = nor.to_crs(epsg = 4326)
shape = pd.concat([shape, nor, add])
shape = shape[~(shape['CNTR_CODE'].isin(['TR', 'RS', 'MK', 'ME', 'AL']))][['NUTS_ID', 'geometry', 'LEVL_CODE']]
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
continents = world.dissolve()

# read shape file by ESPON
# shp0 = gpd.read_file('../../02_Data/01_GIS/01_ESPON Narrow MapKit NUTS2016/nuts-version2016-level0.shp')
# shp1 = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/Data/ESPON MAPS/nuts-version2016-level1.shp')
# shp2 = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/Data/ESPON MAPS/nuts-version2016-level2.shp')
# shp3 = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/Data/ESPON MAPS/nuts-version2016-level3.shp')

##### make the figures

In [ ]:
# set mamp general general aspect 

plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.4
plt.rcParams['grid.color'] = "white"
plt.rcParams['axes.facecolor'] = '#f4f9fa'

In [ ]:
## add the ESPON logo 
plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True

In [ ]:
plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.dpi'] = 80

## 1. Cases

### 1.1 New cases

In [ ]:
## Read the data 
# Cases
cases = pd.read_csv('../../02_Data/02_Pandemic variables/01_Cases/EU_Cases_Weekly_Unstacked.csv')
cases.rename(columns = {"Unnamed: 0": 'nuts_id'}, inplace= True)
cases = cases.melt(id_vars = 'nuts_id', var_name = 'date', value_name = 'cases')
cases['date'] = pd.to_datetime(cases['date']).dt.date
cases = cases.pivot_table(index = 'nuts_id', columns = 'date', values = 'cases')
cases = cases.iloc[:, 3:-9]
cases.fillna(0, inplace = True)
cases

In [ ]:
# cases = cases.fillna(method = 'ffill')

In [ ]:
# turn dataframes into GeoDataFrames 
geo_cases = gpd.GeoDataFrame(pd.merge(cases, shape, left_on = cases.index, right_on = 'NUTS_ID').fillna(np.nan))
geo_cases = pd.concat([geo_cases,fiji])

In [ ]:
# change the CRS to Europe 
geo_cases = geo_cases.to_crs(epsg=3035) 
shape = shape.to_crs(epsg=3035) 
continenets = continents.to_crs(epsg=3035) 
world = world.to_crs(epsg = 3035)
cities = cities.to_crs(epsg = 3035)

In [ ]:
# Selecting only cities we want to see on the plot   
cities['x'] = cities['geometry'].x
cities['y'] = cities['geometry'].y
cities = cities[cities['x'] > 0.23e7]
cities = cities[cities['x'] < 0.67e7]
cities = cities[cities['y'] > 0.1e7]
cities = cities[cities['y'] < 0.545e7]
# Delete these cities from the plot 
cities = cities[~cities['name'].isin(['Beirut', 'Vatican City', 'Cairo', 'Jerusalem', 'Tripoli', 'Zagreb', 'Bratislava'])]

In [ ]:
source="""Regional level: NUTS3 v2016 | Data version 2021.
Source: WHO EPISTAT, NPGEO, mscbs.gob.es.
santé-publique-France, statistichecoronavirus.it.
folkhalsomyndigheten, gov.scot, Korona.gov.sk.
data.gv.at, sso.dk, salute.gov.it."""

In [ ]:
# Creating the bins 
bins = [-1, 5, 100, 1000, 5000, 43040]
label = ['Less than 5', '5-100', '100-1000', '1000 - 500' , 'More than 1000']
for col in geo_cases.iloc[:, :-3]:
    geo_cases[col] = pd.cut(geo_cases[col], bins = bins, labels= label)

In [ ]:
geo_cases

In [ ]:
image_frames = []

# # create the maps 
for i in geo_cases.iloc[:,:-3]:
    fig, ax = plt.subplots(figsize = (12, 12))
    # set out map 
    geo_cases.plot(column = geo_cases[i], ax =ax, cmap = 'RdPu', legend = True, alpha = .4,
              linewidth = 0.5, edgecolor = 'w', categorical = True, missing_kwds={
            "color": "lightgrey",
            "edgecolor": "red",
            "hatch": "///",
            "label": "Missing values"})   
    # add countries boundaries 
    world.query('continent == "Europe"').dissolve().boundary.plot(color = 'black', linewidth = 3, alpha = 0.09, ax = ax, zorder = -1)
    world.plot(ax = ax, color = 'grey', alpha = .1, zorder=0)
    shape.query('LEVL_CODE == 0').boundary.plot(color = 'w', linewidth = 1.4 , zorder  =1, ax =ax)
    world.query('continent == "Africa"').boundary.plot(color = 'w', linewidth = 1.4 , zorder  =1, ax =ax)
    shape.query('LEVL_CODE == 3').boundary.plot(color = 'gray', linewidth = 0.5, zorder=1, ax =ax, alpha = 0.09)
    # set the final limite
    plt.xlim(2.4e6, 6.6e6)
    plt.ylim(0.13e7, 0.545e7)
    # add capitals points 
    cities.plot(ax = ax, color = 'k', markersize = 1)
    for x, y, label in zip(cities.geometry.x, cities.geometry.y, cities.name):
        ax.annotate(label, xy=(x, y), xytext=(1, 2), textcoords="offset points", fontsize = 6.5)
    # Disable scientific representation 
    ax.ticklabel_format(useOffset=False, style='plain')

    # add the graphical-scale 
    scalebar = ScaleBar(0.66, width_fraction=None,
            location= "lower right", box_alpha = 0, length_fraction = 0.08,
                       height_fraction  = 0.003) # 1 pixel = 0.2 meter
    plt.gca().add_artist(scalebar)

    # legend 
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0.24, 0)) 
    leg.get_frame().set_alpha(0)
    leg.set_title('New cases')
    plt.title(f'  Weekly new confirmed COVID-19 cases: {pd.Timestamp(i).month_name()} (week {pd.Timestamp(i).week}) - {i.year}', fontweight = 'light',
            fontsize= 14, loc = 'left', color = '#054ea2')
    # Set outside the figure color 
    fig.patch.set_facecolor('#fbfdfe')
    #  Set source 
    fig.text(0.92, .055, source, ha='right', fontsize = 8)
    # add ESPON's title 
    plt.text(x = 3.5e6, y = 1.4e6, s = u"\u00A9 ESPON EGTC, 2021", fontsize = 10)
    # add bleu lines 
    plt.vlines(x = 2.4e6, ymin = 1e6, ymax = 2e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 2.4e6, ymin = 4.8e6, ymax = 6e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 6.6e6, ymin = 1e6, ymax = 6e6, color = '#054ea2', linewidth = 10)

    plt.hlines(y = 0.13e7, xmin = 5.4e6, xmax = 6.6e6, color = '#054ea2', linewidth = 5)
    plt.hlines(y = 0.545e7, xmin = 2.4e6, xmax = 3.4e6, color = '#054ea2', linewidth = 5)
    # revome axis  
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    # grid params 
    plt.tick_params(bottom = False, left = False)
    plt.grid(True, dash_capstyle = 'projecting')

    # add ESPON logo 
    newax = fig.add_axes([0.1, 0.003,0.14,0.18], anchor='NE', zorder=1)
    newax.imshow(im)
    newax.axis('off')
    plt.show()

    img = ax.get_figure()
    f = io.BytesIO()
    img.savefig(f, format= 'png')
    f.seek(0)
    image_frames.append(PIL.Image.open(f))

In [ ]:
# create a GIF animation 
image_frames[0].save('Animated Maps/Dynamic COVID-19 new cases Map.gif', format = 'GIF', append_images = image_frames[1:],
                    save_all = True, duration = 800,
                     loop = 2)
f.close()

### 1.2 Cumulitave cases

In [ ]:
cases_cumsum = np.cumsum(cases, axis =1)

In [ ]:
# turn dataframes into GeoDataFrames 
cumsum_cases = gpd.GeoDataFrame(pd.merge(cases_cumsum, shape, left_on = cases.index, right_on = 'NUTS_ID').fillna(np.nan))
# cumsum_cases = pd.concat([geo_cases,fiji])

In [ ]:
cumsum_cases

In [ ]:
# Creating the bins 
bins = [0, 50, 100, 1_000, 10_000, 30_000, 60_000,  900_000]
label = ['Less than 50', '50 - 100', 'Less than 1 thousand', '1 - 10 thousands',
         '10 - 300 thousands', '300 - 600 thousands', 'More than 600 thousands']
for col in cumsum_cases.iloc[:, :-3]:
    cumsum_cases[col] = pd.cut(cumsum_cases[col], bins = bins, labels= label)

In [ ]:
# change the CRS to Europe 
cumsum_cases = cumsum_cases.to_crs(epsg=3035) 

In [ ]:
image_frames = []

# # create the maps 
for i in cumsum_cases.iloc[:,:-3]:
    fig, ax = plt.subplots(figsize = (12, 12))
    # set out map 
    cumsum_cases.plot(column = cumsum_cases[i], ax =ax, cmap = 'RdPu', legend = True, alpha = .4,
              linewidth = 0.5, edgecolor = 'w', categorical = True)   
    # add countries boundaries 
    world.query('continent == "Europe"').dissolve().boundary.plot(color = 'black', linewidth = 3, alpha = 0.09, ax = ax, zorder = -1)
    world.plot(ax = ax, color = 'grey', alpha = .1, zorder=0)
    shape.query('LEVL_CODE == 0').boundary.plot(color = 'w', linewidth = 1.4 , zorder  =1, ax =ax)
    world.query('continent == "Africa"').boundary.plot(color = 'w', linewidth = 1.4 , zorder  =1, ax =ax)
    shape.query('LEVL_CODE == 3').boundary.plot(color = 'w', linewidth = 0.5, zorder=1, ax =ax, alpha = 0.09)
    # set the final limite
    plt.xlim(2.4e6, 6.6e6)
    plt.ylim(0.13e7, 0.545e7)
    # add capitals points 
    cities.plot(ax = ax, color = 'k', markersize = 1)
    for x, y, label in zip(cities.geometry.x, cities.geometry.y, cities.name):
        ax.annotate(label, xy=(x, y), xytext=(1, 2), textcoords="offset points", fontsize = 6.5)
    # Disable scientific representation 
    ax.ticklabel_format(useOffset=False, style='plain')

    # add the graphical-scale 
    scalebar = ScaleBar(0.66, width_fraction=None,
            location= "lower right", box_alpha = 0, length_fraction = 0.08,
                       height_fraction  = 0.003) # 1 pixel = 0.2 meter
    plt.gca().add_artist(scalebar)

    # legend 
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0.24, 0)) 
    leg.get_frame().set_alpha(0)
    leg.set_title('Cumulative cases')
    plt.title(f'  Weekly cumulative confirmed COVID-19 cases: {pd.Timestamp(i).month_name()} (week {pd.Timestamp(i).week}) - {i.year}', fontweight = 'light',
            fontsize= 14, loc = 'left', color = '#054ea2')
    # Set outside the figure color 
    fig.patch.set_facecolor('#fbfdfe')
    #  Set source 
    fig.text(0.92, .055, source, ha='right', fontsize = 8)
    # add ESPON's title 
    plt.text(x = 3.5e6, y = 1.4e6, s = u"\u00A9 ESPON EGTC, 2021", fontsize = 10)
    # add bleu lines 
    plt.vlines(x = 2.4e6, ymin = 1e6, ymax = 2e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 2.4e6, ymin = 4.8e6, ymax = 6e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 6.6e6, ymin = 1e6, ymax = 6e6, color = '#054ea2', linewidth = 10)

    plt.hlines(y = 0.13e7, xmin = 5.4e6, xmax = 6.6e6, color = '#054ea2', linewidth = 5)
    plt.hlines(y = 0.545e7, xmin = 2.4e6, xmax = 3.4e6, color = '#054ea2', linewidth = 5)
    # revome axis  
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    # grid params 
    plt.tick_params(bottom = False, left = False)
    plt.grid(True, dash_capstyle = 'projecting')

    # add ESPON logo 
    newax = fig.add_axes([0.1, 0.02,0.14,0.18], anchor='NE', zorder=1)
    newax.imshow(im)
    newax.axis('off')
    plt.show()

    img = ax.get_figure()
    f = io.BytesIO()
    img.savefig(f, format= 'png')
    f.seek(0)
    image_frames.append(PIL.Image.open(f))

In [ ]:
# create a GIF animation 
image_frames[0].save('Animated Maps/Dynamic COVID-19 cumulative cases Map.gif', format = 'GIF', append_images = image_frames[1:],
                    save_all = True, duration = 800,
                     loop = 2)
f.close()

##### 1. cases density 

In [ ]:
d_cases = pd.merge(cases.reset_index(), population, left_on = 'nuts_id', right_on = 'NUTS_ID').drop('nuts_id', axis =1)
d_cases

In [ ]:
density_cases = (d_cases.iloc[:,:-3].div(d_cases['Population'], axis=0) * 10_000).set_index(d_cases['NUTS_ID'])
density_cases

In [ ]:
geo_density_cases = gpd.GeoDataFrame(pd.merge(density_cases, shape, left_on = density_cases.index, right_on = 'NUTS_ID').fillna(np.nan))
geo_density_cases.head(3)

In [ ]:
q = geo_density_cases.round(decimals = 2)
q.columns = [str(col) for col in q.columns]

In [ ]:
q['center'] = q['geometry'].centroid

In [ ]:
q['X'] = q['center'].x
q['Y'] = q['center'].y

In [ ]:
q.drop(['geometry', 'LEVL_CODE', 'NUTS_ID', 'center'], axis =1).to_csv('data.csv')

In [ ]:
(q['center']).to_file('cases_density/points.shp')

In [ ]:
# Creating the bins a
bins = [0, 0.5, 1, 5, 10, 40, 80, 150, 2966]
label = ['< 0.5', '0.5 - 1', '1 - 5', '5 - 10', '',
         '40 - 80', '80 - 150', 'More than 150']
for col in geo_density_cases.iloc[:, :-3]:
    geo_density_cases[col] = pd.cut(geo_density_cases[col], bins = bins, labels= label)

In [ ]:
image_frames = []

# # create the maps 
for i in geo_density_cases.iloc[:,:-3]:
    fig, ax = plt.subplots(figsize = (12, 12))
    # set out map 
    geo_density_cases.plot(column = geo_density_cases[i], ax =ax, cmap = 'RdPu', legend = True, alpha = .4,
              linewidth = 0.5, edgecolor = 'w', categorical = True)   
    # add countries boundaries 
    world.query('continent == "Europe"').dissolve().boundary.plot(color = 'black', linewidth = 3, alpha = 0.09, ax = ax, zorder = -1)
    world.plot(ax = ax, color = 'grey', alpha = .1, zorder=0)
    shape.query('LEVL_CODE == 0').boundary.plot(color = 'w', linewidth = 1.4 , zorder  =1, ax =ax)
    world.query('continent == "Africa"').boundary.plot(color = 'w', linewidth = 1.4 , zorder  =1, ax =ax)
    shape.query('LEVL_CODE == 3').boundary.plot(color = 'w', linewidth = 0.5, zorder=1, ax =ax, alpha = 0.09)
    # set the final limite
    plt.xlim(2.4e6, 6.6e6)
    plt.ylim(0.13e7, 0.545e7)
    # add capitals points 
    cities.plot(ax = ax, color = 'k', markersize = 1)
    for x, y, label in zip(cities.geometry.x, cities.geometry.y, cities.name):
        ax.annotate(label, xy=(x, y), xytext=(1, 2), textcoords="offset points", fontsize = 6.5)
    # Disable scientific representation 
    ax.ticklabel_format(useOffset=False, style='plain')

    # add the graphical-scale 
    scalebar = ScaleBar(0.66, width_fraction=None,
            location= "lower right", box_alpha = 0, length_fraction = 0.08,
                       height_fraction  = 0.003) # 1 pixel = 0.2 meter
    plt.gca().add_artist(scalebar)

    # legend 
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0.24, 0)) 
    leg.get_frame().set_alpha(0)
    leg.set_title('Cases density')
    plt.title(f'  Weekly COVID-19 cases density: {pd.Timestamp(i).month_name()} (week {pd.Timestamp(i).week}) - {i.year}', fontweight = 'light',
            fontsize= 14, loc = 'left', color = '#054ea2')
    # Set outside the figure color 
    fig.patch.set_facecolor('#fbfdfe')
    #  Set source 
    fig.text(0.92, .055, source, ha='right', fontsize = 8)
    # add ESPON's title 
    plt.text(x = 3.5e6, y = 1.4e6, s = u"\u00A9 ESPON EGTC, 2021", fontsize = 10)
    # add bleu lines 
    plt.vlines(x = 2.4e6, ymin = 1e6, ymax = 2e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 2.4e6, ymin = 4.8e6, ymax = 6e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 6.6e6, ymin = 1e6, ymax = 6e6, color = '#054ea2', linewidth = 10)

    plt.hlines(y = 0.13e7, xmin = 5.4e6, xmax = 6.6e6, color = '#054ea2', linewidth = 5)
    plt.hlines(y = 0.545e7, xmin = 2.4e6, xmax = 3.4e6, color = '#054ea2', linewidth = 5)
    # revome axis  
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    # grid params 
    plt.tick_params(bottom = False, left = False)
    plt.grid(True, dash_capstyle = 'projecting')

    # add ESPON logo 
    newax = fig.add_axes([0.1, 0.02,0.14,0.18], anchor='NE', zorder=1)
    newax.imshow(im)
    newax.axis('off')
    plt.show()

    img = ax.get_figure()
    f = io.BytesIO()
    img.savefig(f, format= 'png')
    f.seek(0)
    image_frames.append(PIL.Image.open(f))

In [ ]:
# create a GIF animation 
image_frames[0].save('Animated Maps/Dynamic COVID-19 case density Map.gif', format = 'GIF', append_images = image_frames[1:],
                    save_all = True, duration = 800,
                     loop = 2)
f.close()

## 2. Deaths

### 2.1 New deaths 

In [7]:
# Cases
deaths = pd.read_csv('../../02_Data/02_Pandemic variables/02_Deaths/EU_Deaths_Weekly_Unstacked.csv')

deaths.rename(columns = {"Unnamed: 0": 'nuts_id'}, inplace= True)
deaths = deaths.melt(id_vars = 'nuts_id', var_name = 'date', value_name = 'deaths')
deaths['date'] = pd.to_datetime(deaths['date']).dt.date
deaths = deaths.pivot_table(index = 'nuts_id', columns = 'date', values = 'deaths')
deaths = deaths.iloc[:, 3:-9]
deaths.fillna(method = 'ffill', inplace = True)
deaths.fillna(0, inplace= True)
deaths 

date,2020-01-26,2020-02-02,2020-02-08,2020-02-16,2020-02-23,2020-03-05,2020-03-15,2020-03-22,2020-03-29,2020-04-10,...,2021-10-01,2021-10-10,2021-10-17,2021-10-24,2021-10-31,2021-11-04,2021-11-07,2021-11-14,2021-11-21,2021-11-28
nuts_id,,,,,,,,,,,,,,,,,,,,,
AT111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0
AT112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0,1.0,3.0
AT113,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,5.0,1.0,1.0,0.0,0.0,7.0,0.0,2.0,1.0,0.0
AT121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,...,15.0,1.0,1.0,3.0,3.0,5.0,0.0,5.0,8.0,2.0
AT122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,...,14.0,0.0,5.0,2.0,5.0,14.0,0.0,6.0,11.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UKJ,0.0,0.0,18.0,0.0,0.0,417.0,16.0,93.0,289.0,24.0,...,1277.0,58.0,67.0,77.0,115.0,14.0,16.0,96.0,96.0,91.0
UKK,0.0,0.0,1.0,0.0,0.0,185.0,3.0,25.0,93.0,8.0,...,326.0,52.0,59.0,76.0,94.0,9.0,5.0,78.0,74.0,74.0
UKL,0.0,0.0,17.0,0.0,0.0,130.0,0.0,17.0,77.0,29.0,...,284.0,51.0,67.0,68.0,69.0,8.0,4.0,63.0,53.0,40.0


In [8]:
# turn dataframes into GeoDataFrames 
geo_deaths = gpd.GeoDataFrame(pd.merge(deaths, shape, left_on = deaths.index, right_on = 'NUTS_ID').fillna(0))
# geo_deaths = pd.concat([geo_deaths,fiji])

In [ ]:
for col in geo_deaths.iloc[:, :-3]:
    geo_deaths[col] = pd.to_numeric(geo_deaths[col]).astype(int)

In [10]:
geo_deaths['center'] = geo_deaths['geometry'].centroid

C:\Users\Mohamed\AppData\Local\Temp/ipykernel_2728/2255727799.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_deaths['center'] = geo_deaths['geometry'].centroid


In [37]:
s = gpd.GeoDataFrame(geo_deaths[['center']])
type(s)

geopandas.geodataframe.GeoDataFrame

In [47]:
geo_deaths['center'].x

0      16.480591
1      16.756576
2      16.256577
3      15.000778
4      15.899147
         ...    
491    -0.537925
492    -3.129929
493    -3.765118
494    -4.190438
495    -6.694720
Length: 496, dtype: float64

In [48]:
geo_deaths['x'] = geo_deaths['center'].x
geo_deaths['y'] = geo_deaths['center'].y

In [57]:
q = geo_deaths.drop(['NUTS_ID', 'LEVL_CODE', 'center', 'geometry'], axis =1)

In [60]:
q.columns = [str(i) for i in q.columns]

In [69]:
q.to_excel('points/points.xlsx', index = False)

In [20]:
geo_deaths.drop(['NUTS_ID', 'geometry', 'LEVL_CODE'], axis =1).to_file('points/points.shp')

TypeError: Cannot interpret '<geopandas.array.GeometryDtype object at 0x00000279D40607C0>' as a data type

In [14]:
geo_deaths[['NUTS_ID', 'geometry']].to_file('death_density/shape.shp')

In [ ]:
# Creating the bins 
bins = [-1, 1, 5, 20, 50 ,100 , 1000, 2904]
label = [' 1', '< 5', '< 20', '< 50', '< 100', '< 1000', '> 1000']
for col in geo_deaths.iloc[:, :-3]:
    geo_deaths[col] = pd.cut(geo_deaths[col], bins = bins, labels= label)

In [ ]:
image_frames = []

# # create the maps 
for i in geo_deaths.iloc[:,:-3]:
    fig, ax = plt.subplots(figsize = (12, 12))
    # set out map 
    geo_deaths.plot(column = geo_deaths[i], ax =ax, cmap = 'RdPu', legend = True, alpha = .4,
              linewidth = 0.5, edgecolor = 'w', categorical = True)   
    # add countries boundaries 
    world.query('continent == "Europe"').dissolve().boundary.plot(color = 'black', linewidth = 3, alpha = 0.09, ax = ax, zorder = -1)
    world.plot(ax = ax, color = 'grey', alpha = .1, zorder=0)
    shape.query('LEVL_CODE == 0').boundary.plot(color = 'w', linewidth = 1.4 , zorder  =1, ax =ax)
    world.query('continent == "Africa"').boundary.plot(color = 'w', linewidth = 1.4 , zorder  =1, ax =ax)
    shape.query('LEVL_CODE == 3').boundary.plot(color = 'gray', linewidth = 0.5, zorder=1, ax =ax, alpha = 0.09)
    # set the final limite
    plt.xlim(2.4e6, 6.6e6)
    plt.ylim(0.13e7, 0.545e7)
    # add capitals points 
    cities.plot(ax = ax, color = 'k', markersize = 1)
    for x, y, label in zip(cities.geometry.x, cities.geometry.y, cities.name):
        ax.annotate(label, xy=(x, y), xytext=(1, 2), textcoords="offset points", fontsize = 6.5)
    # Disable scientific representation 
    ax.ticklabel_format(useOffset=False, style='plain')

    # add the graphical-scale 
    scalebar = ScaleBar(0.66, width_fraction=None,
            location= "lower right", box_alpha = 0, length_fraction = 0.08,
                       height_fraction  = 0.003) # 1 pixel = 0.2 meter
    plt.gca().add_artist(scalebar)

    # legend 
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0.24, 0)) 
    leg.get_frame().set_alpha(0)
    leg.set_title('New deaths')
    plt.title(f'  Weekly new confirmed COVID-19 deaths: {pd.Timestamp(i).month_name()} (week {pd.Timestamp(i).week}) - {i.year}', fontweight = 'light',
            fontsize= 14, loc = 'left', color = '#054ea2')
    # Set outside the figure color 
    fig.patch.set_facecolor('#fbfdfe')
    #  Set source 
    fig.text(0.92, .055, source, ha='right', fontsize = 8)
    # add ESPON's title 
    plt.text(x = 3.5e6, y = 1.4e6, s = u"\u00A9 ESPON EGTC, 2021", fontsize = 10)
    # add bleu lines 
    plt.vlines(x = 2.4e6, ymin = 1e6, ymax = 2e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 2.4e6, ymin = 4.8e6, ymax = 6e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 6.6e6, ymin = 1e6, ymax = 6e6, color = '#054ea2', linewidth = 10)

    plt.hlines(y = 0.13e7, xmin = 5.4e6, xmax = 6.6e6, color = '#054ea2', linewidth = 5)
    plt.hlines(y = 0.545e7, xmin = 2.4e6, xmax = 3.4e6, color = '#054ea2', linewidth = 5)
    # revome axis  
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    # grid params 
    plt.tick_params(bottom = False, left = False)
    plt.grid(True, dash_capstyle = 'projecting')

    # add ESPON logo 
    newax = fig.add_axes([0.1, 0.02,0.14,0.18], anchor='NE', zorder=1)
    newax.imshow(im)
    newax.axis('off')
    plt.show()

    img = ax.get_figure()
    f = io.BytesIO()
    img.savefig(f, format= 'png')
    f.seek(0)
    image_frames.append(PIL.Image.open(f))

In [ ]:
# create a GIF animation 
image_frames[0].save('Animated Maps/Dynamic COVID-19 new deaths Map.gif', format = 'GIF', append_images = image_frames[1:],
                    save_all = True, duration = 800,
                     loop = 2)
f.close()

## 3. Exc. Mortality 

### 1.2 Exc. Mortality 

In [ ]:
mortality = pd.read_excel('../../02_Data/02_Pandemic variables/03_Exc. Mortality/COVID_Excess_Mortality_v220103.xlsx').drop(['Country',
                                                                                                                            'NUTS_level',                                                                                                                             'NUTS level 3',
                                                                                                                            'Pop_2020_01_01'], axis =1)
mortality = mortality.melt(id_vars = 'NUTS_ID_2021', var_name = "date", value_name = 'mortality')
mortality['date'] = pd.to_datetime(mortality['date']).dt.date
mortality = mortality.pivot_table(index = 'NUTS_ID_2021', columns = 'date', values ='mortality')
mortality 

In [ ]:
# turn dataframes into GeoDataFrames 
geo_mortality = gpd.GeoDataFrame(pd.merge(mortality, shape, left_on = mortality.index, right_on = 'NUTS_ID').fillna(np.nan))

In [ ]:
# Creating the bins 
bins = [-760,  -100 , -20, -5, 5, 20, 200,  4400]


label = ['< 100' ,'< 20', '< 5' ,'[-5 / 5]', '> 5', '> 20', '> 200']
for col in geo_mortality.iloc[:, :-3]:
    geo_mortality[col] = pd.cut(geo_mortality[col], bins = bins, labels= label)

In [ ]:
source="""Regional level: NUTS3 v2021 | Data version 2021.
Source: EuroStat database, NPGEO, mscbs.gob.es.
santé-publique-France, statistichecoronavirus.it.
folkhalsomyndigheten, gov.scot, Korona.gov.sk.
data.gv.at, sso.dk, salute.gov.it."""

In [ ]:
image_frames = []

# # create the maps 
for i in geo_mortality.iloc[:,:-3]:
    fig, ax = plt.subplots(figsize = (12, 12))
    # set out map 
    geo_mortality.plot(column = geo_mortality[i], ax =ax, cmap = 'coolwarm', alpha = .4,
              linewidth = 0.5, edgecolor = 'w', categorical = True, legend = True)   
    # add countries boundaries 
    world.query('continent == "Europe"').dissolve().boundary.plot(color = 'black', linewidth = 3, alpha = 0.09, ax = ax, zorder = -1)
    world.plot(ax = ax, color = 'grey', alpha = .1, zorder=0)
    shape.query('LEVL_CODE == 0').boundary.plot(color = 'w', linewidth = 1.4 , zorder  =1, ax =ax)
    world.query('continent == "Africa"').boundary.plot(color = 'w', linewidth = 1.4 , zorder  =1, ax =ax)
    shape.query('LEVL_CODE == 3').boundary.plot(color = 'gray', linewidth = 0.5, zorder=1, ax =ax, alpha = 0.09)
    # set the final limite
    plt.xlim(2.4e6, 6.6e6)
    plt.ylim(0.13e7, 0.545e7)
    # add capitals points 
    cities.plot(ax = ax, color = 'k', markersize = 1)
    for x, y, label in zip(cities.geometry.x, cities.geometry.y, cities.name):
        ax.annotate(label, xy=(x, y), xytext=(1, 2), textcoords="offset points", fontsize = 6.5)
    # Disable scientific representation 
    ax.ticklabel_format(useOffset=False, style='plain')

    # add the graphical-scale 
    scalebar = ScaleBar(0.66, width_fraction=None,
            location= "lower right", box_alpha = 0, length_fraction = 0.08,
                       height_fraction  = 0.003) # 1 pixel = 0.2 meter
    plt.gca().add_artist(scalebar)

    # legend 
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0.24, 0)) 
    leg.get_frame().set_alpha(0)
    leg.set_title('Excess mortality')
    plt.title(f'  Weekly excess mortality: {pd.Timestamp(i).month_name()} (week {pd.Timestamp(i).week}) - {i.year}', fontweight = 'light',
            fontsize= 14, loc = 'left', color = '#054ea2')
    # Set outside the figure color 
    fig.patch.set_facecolor('#fbfdfe')
    #  Set source 
    fig.text(0.92, .055, source, ha='right', fontsize = 8)
    # add ESPON's title 
    plt.text(x = 3.5e6, y = 1.4e6, s = u"\u00A9 ESPON EGTC, 2021", fontsize = 10)
    # add bleu lines 
    plt.vlines(x = 2.4e6, ymin = 1e6, ymax = 2e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 2.4e6, ymin = 4.8e6, ymax = 6e6, color = '#054ea2', linewidth = 10)
    plt.vlines(x = 6.6e6, ymin = 1e6, ymax = 6e6, color = '#054ea2', linewidth = 10)

    plt.hlines(y = 0.13e7, xmin = 5.4e6, xmax = 6.6e6, color = '#054ea2', linewidth = 5)
    plt.hlines(y = 0.545e7, xmin = 2.4e6, xmax = 3.4e6, color = '#054ea2', linewidth = 5)
    # revome axis  
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    # grid params 
    plt.tick_params(bottom = False, left = False)
    plt.grid(True, dash_capstyle = 'projecting')

    # add ESPON logo 
    newax = fig.add_axes([0.11, 0.02,0.14,0.18], anchor='NE', zorder=1)
    newax.imshow(im)
    newax.axis('off')
    plt.show()

    img = ax.get_figure()
    f = io.BytesIO()
    img.savefig(f, format= 'png')
    f.seek(0)
    image_frames.append(PIL.Image.open(f))

In [ ]:
# create a GIF animation 
image_frames[0].save('Animated Maps/Dynamic exces mortality Map.gif', format = 'GIF', append_images = image_frames[1:],
                    save_all = True, duration = 800,
                     loop = 2)
f.close()